In [ ]:
using ModelingToolkit, MethodOfLines, DifferentialEquations, Plots, DomainSets


xleft::Float64 = 0.0
xright::Float64 = 1.0
t0::Float64 = 0.0
T::Float64 = 20.0
N = 100
order = 2
step = (xright - xleft)/N
grid = collect(xleft:step:xright)

#Definition of constants (placeholders):
gamma::Float64 = 0
gamma_3::Float64 = 10
c::Float64 = 1
omega_d::Float64 = 10

#Define parameters and variables (note: u(..) is valid as a symbolic function notation):
@parameters x t
@variables u(..)

#Define differential operators:
Dt = Differential(t)
Dx = Differential(x)
Dtt = Differential(t)^2
Dxx = Differential(x)^2

#Define source function (placeholder constant spatial term):
source_f(x, t) = exp(-40(x)^2) * sin(omega_d*t)

#Define initial condition (placeholder sinusoidal):
#u0(x, t) = exp(-(35*pi)*(x-0.5)^2)
u0(x, t) = 0.01 * sin(3.14 * x)
v0(x) = 0


#Define equation (with and without cubic damping term):
equation = [Dtt(u(x,t)) + gamma*Dt(u(x,t)) + gamma_3*(Dt(u(x,t)))^3 ~ c^2*Dxx(u(x,t)) + source_f(x, t)]
#equation = [Dtt(u(x,t)) ~ c^2*Dxx(u(x,t))]


#Define domains:
domains = [x ∈ Interval(xleft, xright), t ∈ Interval(t0, T)]

#Define boundary conditions:
bound_conditions = [u(x, 0) ~ u0(x, 0),
                     u(xleft, t) ~ 0,
                     u(xright, t) ~ 0,
                     Dt(u(x, 0)) ~ v0(x)]

In [ ]:
#Create PDE system and assign identifier (order matters: eq, bcs, domains, paramers, variables; format parameters and variables as lists, provide arguments whenever they exist):
@named pde_sys = PDESystem(equation, bound_conditions, domains, [x, t], [u(x,t)])

#Define a discretization (first argument specifies parameters to discretize, second indicates parameters to keep continuous, third provides the approximation order):
discretization = MOLFiniteDifference([x=>grid], t, approx_order=order)

#Proceed with discretization:
@time eq_discretized = discretize(pde_sys,discretization)

#Solve the resulting system (specified solver dictates whether or not time marching is used, as well as the method: Euler(), ImplicitEuler(), or Trapezoidal(), all with given time steps):
#Warning: if the time step is not explicitly given, solver may default to actual method of lines, instead of time marching
solution= solve(eq_discretized, ImplicitEuler(), dt=1e-2, saveat=0.01)



In [ ]:
#Plot results:
discrete_t = solution.t

sol_u = solution[u(x, t)]

Nx = length(grid)
Nt = length(solution.t)

#gr()

animation = @animate for k in 1:Nt
    plot(grid, sol_u[1:end, k],
         title="t = $(solution.t[k])", xlabel="x", ylabel="u(x,t)", ylim = (-1, 1))
end

#display(animation)

gif(animation, "FD.gif", fps = 60)